In [2]:
import time

# visualizatoin
import matplotlib.pyplot as plt

# data wrangling
import pandas as pd
import numpy as np

# data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer

# learning
from sklearn.linear_model import LogisticRegression

In [3]:
# description
description = pd.read_csv('data/WiDS_Datathon_2020_Dictionary.csv')
description_dict = description.set_index('Variable Name').to_dict(orient='index')
# data
df = pd.read_csv('data/training_v2.csv')

df.head()

,encounter_id,patient_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,66154,25312,118,0,68.0,22.73,0,Caucasian,M,180.3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular
1,114252,59342,81,0,77.0,27.42,0,Caucasian,F,160.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory
2,119783,50777,118,0,25.0,31.95,0,Caucasian,F,172.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic
3,79267,46918,118,0,81.0,22.64,1,Caucasian,F,165.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
4,92056,34377,33,0,19.0,NaN,0,Caucasian,M,188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma


In [21]:
# explanation cell

description_dict['icu_type']

{'Category': 'demographic',
 'Unit of Measure': 'None',
 'Data Type': 'string',
 'Description': 'A classification which indicates the type of care the unit is capable of providing',
 'Example': 'Neurological ICU'}

# Remove identifiers and results from df

In [7]:
test_size = 0.2 # proportion for train versus test+val split
val_size = 0.5 # proportion for test versus val split
random_state = 42  # random state is used to set a seed for randomness, which is only relevant for reproducibility purposes

# save features
X = df.copy().drop(['hospital_death', 'patient_id', 'encounter_id', 'hospital_id', 'icu_id', # drop identifiers
                    'apache_4a_hospital_death_prob', 'apache_4a_icu_death_prob', # drop APACHE scores
                    'apache_2_bodysystem'], # drop because of similarity with apache_3j_bodysystem
                   axis=1)
# save target variable
y = df['hospital_death'].copy()
# save APACHE scores for later evaluation on train / test / validation data
y_apache = df['apache_4a_hospital_death_prob'].copy()

0        0
1        0
2        0
3        0
4        0
        ..
91708    0
91709    0
91710    0
91711    0
91712    0
Name: hospital_death, Length: 91713, dtype: int64

# Test-Validation-Train split

In [9]:
""" SPLIT DATA SET """
# split the dataset into train and test+validation set
(
    X_train,
    X_test,
    y_train,
    y_test,
    y_apache_train,
    y_apache_test,
    ) = train_test_split(X, y, y_apache,
                         test_size=test_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        )
# split the test set into test + validation set
(
    X_val,
    X_test,
    y_val,
    y_test,
    y_apache_val,
    y_apache_test,
    ) = train_test_split(X_test, y_test, y_apache_test,
                         test_size=val_size, # used for testing and validation
                         random_state=random_state # for reproducibility
                        )

# explore categorical values, to see if we can eliminate categories that we don't need in the one hot encoding

Note for future: I believe some of the icu_types have overlap, for example, CCI-CTICU and CTICU

In [17]:
df.loc[:, ["ethnicity", "gender", 'hospital_admit_source', "icu_admit_source", 'icu_stay_type', 'icu_type', 'readmission_status']]["icu_admit_source"].unique()

array(['Floor', 'Accident & Emergency', 'Operating Room / Recovery',
       'Other Hospital', 'Other ICU', nan], dtype=object)

In [32]:
df.loc[:, ["ethnicity", "gender", 'hospital_admit_source', "icu_admit_source", 'icu_stay_type', 'icu_type', 'readmission_status']].groupby("icu_type").value_counts()

icu_type   ethnicity  gender  hospital_admit_source  icu_admit_source           icu_stay_type  readmission_status
CCU-CTICU  Caucasian  M       Emergency Department   Accident & Emergency       admit          0                     1126
                      F       Emergency Department   Accident & Emergency       admit          0                      850
                      M       Operating Room         Operating Room / Recovery  admit          0                      549
                              Direct Admit           Accident & Emergency       admit          0                      374
                      F       Operating Room         Operating Room / Recovery  admit          0                      277
                                                                                                                     ... 
SICU       Caucasian  M       Direct Admit           Other ICU                  transfer       0                        1
                                


# remove missing values

In [ ]:
max_missing = 0.8  # maximum percentage of missing values for a column to be dropped


# TODO: instead of removing missing values, we can mark them as missing using sklearn.impute MissingIndicator
"""MISSING VALUES"""
# drop columns with many missing values
missing = X_train.isna().sum() > max_missing * len(X_train)
missing = missing[missing].index
X_train = X_train.drop(missing, axis=1)
X_val = X_val.drop(missing, axis=1)
X_test = X_test.drop(missing, axis=1)

# Preprocessing pipeline:
Turn categories into true = 1, false = 0

In [ ]:
"""FURTHER PROCESSING PIPELINE"""
# define pre-processing steps for numerical features
# TODO: mean strategy is probably not the best
num_transformer = Pipeline(steps=[("constant", VarianceThreshold()), # remove constant features
                                  ("imputer", SimpleImputer(strategy="mean")),
                                 ])


# TODO: same here, improvements can be made
# define preprocessing steps for categorical features
cat_transformer = Pipeline(steps=[("encoder", OneHotEncoder(drop='first', sparse=False, handle_unknown="ignore"))])



# create preprocessing pipeline
prep_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_transformer, make_column_selector(dtype_exclude=object)), # apply to columns NOT of type object (int or float)
        ('cat', cat_transformer, make_column_selector(dtype_include=object)) # apply to columns of type object
    ])
# pipeline
prep_pipeline.fit(X_train, y_train)
display(prep_pipeline) # disply preprocessing pipeline

# apply pipeline

In [ ]:
# transform data sets
X_train = pd.DataFrame(prep_pipeline.transform(X_train), columns=prep_pipeline.get_feature_names_out())
X_val = pd.DataFrame(prep_pipeline.transform(X_val), columns=prep_pipeline.get_feature_names_out())
X_test = pd.DataFrame(prep_pipeline.transform(X_test), columns=prep_pipeline.get_feature_names_out())